# Excercises 
# 1. Tune the network
Run the experiment below, explore the different parameters (see suggestions below) and study the result with tensorboard. 
Make a single page (1 a4) report of your findings. Use your visualisation skills to communicate your most important findings.

In [9]:
from mads_datasets import DatasetFactoryProvider, DatasetType

from mltrainer.preprocessors import BasePreprocessor
from mltrainer import imagemodels, Trainer, TrainerSettings, ReportTypes, metrics

import torch.optim as optim
from torch import nn
from tomlserializer import TOMLSerializer
import torch

We will be using `tomlserializer` to easily keep track of our experiments, and to easily save the different things we did during our experiments.
It can export things like settings and models to a simple `toml` file, which can be easily shared, checked and modified.

First, we need the data. 

In [10]:
fashionfactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
preprocessor = BasePreprocessor()
streamers = fashionfactory.create_datastreamer(batchsize=64, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
trainstreamer = train.stream()
validstreamer = valid.stream()

2026-01-16 13:02:26.909 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/max/.cache/mads_datasets/fashionmnist
2026-01-16 13:02:26.910 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/max/.cache/mads_datasets/fashionmnist/fashionmnist.pt


In [44]:
next(trainstreamer)[0][0][0].shape

torch.Size([28, 28])

We need a way to determine how well our model is performing. We will use accuracy as a metric.

In [45]:
accuracy = metrics.Accuracy()

We will use a very basic model: a model with three linear layers.

In [92]:
class NeuralNetwork(nn.Module):
    def __init__(self,
        num_classes: int,
        units1: int,
        units2: int,
        units3: int,
        # units4: int,
        # units5: int,
        dropout: float
    ) -> None:
        super().__init__()
        # self.num_classes = num_classes
        # self.units1 = units1
        # self.units2 = units2
        # self.units3 = units3
        # self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(1, 3, kernel_size=5, stride=2),
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.Conv2d(3, 5, kernel_size=3),
            nn.BatchNorm2d(5),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(500, units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.ReLU(),
            nn.Linear(units2, units3),
            nn.BatchNorm1d(units3),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(units3, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [93]:
import torch

loss_fn = torch.nn.CrossEntropyLoss()

settings = TrainerSettings(
    epochs=16,
    metrics=[accuracy],
    logdir="modellogs_normalisation",
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.TENSORBOARD, ReportTypes.TOML],
)

model = NeuralNetwork(
    num_classes=10,
    units1=256,
    units2=128,
    units3=64,
    dropout=0.0
)

trainer = Trainer(
    model=model,
    settings=settings,
    loss_fn=loss_fn,
    optimizer=optim.Adam,
    traindataloader=trainstreamer,
    validdataloader=validstreamer,
    scheduler=optim.lr_scheduler.ReduceLROnPlateau
)
trainer.loop()


2026-01-16 14:35:49.092 | INFO     | mltrainer.trainer:dir_add_timestamp:24 - Logging to modellogs_normalisation/20260116-143549
2026-01-16 14:35:49.093 | INFO     | mltrainer.trainer:__init__:68 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|██████████| 937/937 [00:03<00:00, 248.93it/s]
2026-01-16 14:35:53.096 | INFO     | mltrainer.trainer:report:209 - Epoch 0 train 0.4803 test 0.4383 metric ['0.8309']
100%|██████████| 937/937 [00:03<00:00, 240.39it/s]
2026-01-16 14:35:57.242 | INFO     | mltrainer.trainer:report:209 - Epoch 1 train 0.3396 test 0.3799 metric ['0.8582']
100%|██████████| 937/937 [00:03<00:00, 242.59it/s]
2026-01-16 14:36:01.371 | INFO     | mltrainer.trainer:report:209 - Epoch 2 train 0.3019 test 0.3450 metric ['0.8708']
100%|██████████| 937/937 [00:03<00:00, 250.23it/s]
2026-01-16 14:36:05.356 | INFO     | mltrainer.trainer:report:209 - Epoch 3 train 0.2687 test 0.3489 metric ['0.8768']
2026-01-16 14:36:05.357 | INFO     | mltrai